In [1]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

In [2]:
import minsearch

/workspaces/LLM-_Zoomcamp_CaroVergara/LLM_01_intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [3]:
import json

In [4]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
boost = {'question':3.0, 'section': 0.5}

results = index.search(
    query = q,
    boost_dict = boost,
    num_results = 5
)


In [11]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [12]:
from openai import OpenAI

In [13]:
client = OpenAI()

In [14]:
def search(query):
    boost = {'question':3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict = {'course':'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 10
    )

    return results

In [15]:
def build_prompt(query,search_results):
    prompt_template = """
You´re a course teaching assitant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question = query, context=context).strip()

    return prompt
    

In [16]:
def llm(prompt):
    response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role":"user", "content":prompt}]
)

    return response.choices[0].message.content

In [17]:
#query = 'the course already started. Can I still enroll?'
query = 'how i can run kafka?'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

In [18]:
answer

"To run Kafka, you need to follow certain steps depending on your setup. Here are the relevant options from the context:\n\n1. **For Python Kafka**: \n   - Create a virtual environment and install the necessary packages. Here's a quick guide:\n     ```bash\n     python -m venv env          # Create virtual environment\n     source env/bin/activate     # Activate the virtual environment\n     pip install -r ../requirements.txt  # Install dependencies\n     ```\n     Note: On Windows, use `env/Scripts/activate` to activate the virtual environment. Ensure that Docker images are up and running.\n\n2. **For Java Kafka**:\n   - If using Java, you can run a Kafka producer or consumer in your terminal by navigating to your project directory and executing:\n     ```bash\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\n3. **Fixing Errors**:\n   - If you encounter `kafka.errors.NoBrokersAvailable`, check if your Kafka broker Docker

In [19]:
query = 'how i can run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [20]:
rag(query)

"To run Kafka, you'll need to set up a virtual environment and ensure all dependencies are installed. For Python, the suggested steps are:\n\n1. **Create a virtual environment**:\n   - Run `python -m venv env` to create the environment.\n   - Activate it with `source env/bin/activate` (for MacOS/Linux) or `env\\Scripts\\activate` (for Windows).\n\n2. **Install necessary packages**:\n   - While in the activated virtual environment, use `pip install -r ../requirements.txt` to install the necessary packages.\n\n3. **Running Kafka components**:\n   - If you encounter `kafka.errors.NoBrokersAvailable`, ensure that your Kafka broker docker container is running. Use `docker ps` to check the running containers.\n   - If not running, navigate to the docker compose yaml file folder and execute `docker compose up -d` to start all instances.\n\nRemember, Docker images should be running before executing Python files using Kafka."

In [21]:
rag('the course already started. Can I still enroll?')

"Yes, you can still enroll in the course after it has started. Even if you don't register, you are eligible to submit the homework. However, keep in mind that there are deadlines for submitting the final projects, so make sure not to leave everything for the last minute."

In [22]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [24]:
from elasticsearch import Elasticsearch

In [25]:
es_client = Elasticsearch('http://localhost:9200')

In [26]:
es_client.info()

ObjectApiResponse({'name': '5d14e0fef7ff', 'cluster_name': 'docker-cluster', 'cluster_uuid': '8oqwLEuTS_CFyUV2OamRsw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [27]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [28]:
from tqdm.auto import tqdm

In [29]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [32]:
query = 'I just discovered the course. Can I still join it?'

In [44]:
def elastic_search(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response ['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [45]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [46]:
rag(query)

'Yes, you can still join the course even if you discovered it after the start date. You are eligible to submit the homework. However, be mindful of the deadlines for submitting the final projects. Make sure not to leave everything until the last minute.'